# Prototype Aus-wide saltmarsh

## Conceptual workflow 

The covariate data will be flexible, starting with Landsat derived metrics

For the moment, to develop performance, use a once-off geomoedian to represent an agreed time period. Something like:  
    
    1. Decide on a time period, a 2 to 3 year period (2019-2021? - if we stick to post-2015 we can integrate Sentinel-2 down the track more easily)  
    2. Sample from the whole Australia intertidal data set to get wetland vs. not wetland [but for the moment skip to saltmarsh vs land/water?]  
    3. Fit a ML model based on ALL the Aus-wide data (e.g. RF or SVM etc.)  
    4. Predict out to some known locations (eventually guided runs across the whole coastline)
    5. Explore and validate  


***Cummulaive Q's:***
- RE the `query` dictionary passed to the loading function or loading helper: does the `output_crs` only apply to output after the processing subsquent to the load, or does it always apply to every time-step found in the collection?  
- what happens when you stop execution while dask chunks are processing/queued (seems not to like that...)?  
- for lazy eval, is it quicker to `.load()`/`.compute()` just the individual bands, several at one, or the whole lot?  And if several at once, how to do that via the slot method (i.e. `lazy_dat.blue.load()` --> `lazy_day.some_bands.load()`  
- where does the datacube connection need to go? functionalised as needed, or just up top??
- what EPSG t owork with in general for various Aus-wide prediciton windows?
- what does the `dc = datacube.Datacube(app='Classify_satellite_data')` call actually do?
- what is the current standard for pixel-based cloud filtering (for LS8 and S2)


** critical Q's **
- once settled on a larger area, should we export the temporal composites/metrics?
- is it possible to do a lazy training sample (like for a geo-median) but for other computed metrics? (i.e. do they need to be pre-computed, or can they also be only computed where a trianing point is). THe reason being is that we can fit and save the FULL model, but only predict to where we need
- prefer to make multiple scripts that generate progressive outputs (e.g. layer metrics, fitted model, intermediate classifications) OR use flags iwthin one script to skip and load if already done?
- how do i get the training data sampling process to keep more than one column? (i.e. to add further covariates or class labels, or to say keep an ID column to enable join-merge after)
- how to get medians for dask array? how do you usually do geomedians - explicitly with a dedicated function?
- 

## Load packages/modules

In [1]:
%matplotlib inline

import os
import datacube
import numpy as np
import xarray as xr
import subprocess as sp
import geopandas as gpd
import pandas as pd
from odc.io.cgroups import get_cpu_quota
from datacube.utils.geometry import assign_crs

import sys
sys.path.insert(1, '../../Tools/')
from dea_tools.datahandling import load_ard
from dea_tools.plotting import map_shapefile, rgb, display_map
from dea_tools.bandindices import calculate_indices
from dea_tools.classification import collect_training_data
from dea_tools.classification import predict_xr
from dea_tools.dask import create_local_dask_cluster

import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve, auc, balanced_accuracy_score

import dask.array as da

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

# connect to the datacube
dc = datacube.Datacube(app='aust_saltmarsh')

### get ncpus to work with

In [2]:
ncpus = round(get_cpu_quota())
print('ncpus = ' + str(ncpus))

ncpus = 2


In [3]:
print(xr.__version__)

0.19.0
